In [1]:
!pip install -q diffusers transformers accelerate opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00


In [2]:
import torch
import gc
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from transformers import DPTFeatureExtractor, DPTForDepthEstimation

In [3]:
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11f1p_sd15_depth", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,
).to("cuda")

pipe.enable_vae_slicing()
pipe.enable_attention_slicing()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/945 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [4]:
depth_feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-hybrid-midas")
depth_model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas").to("cuda")

def get_depth_map(image: Image.Image):
    image = image.resize((384, 384))
    inputs = depth_feature_extractor(images=image, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = depth_model(**inputs)
        depth = outputs.predicted_depth[0]
    depth = depth.cpu().numpy()
    depth = (depth - depth.min()) / (depth.max() - depth.min()) * 255
    return Image.fromarray(depth.astype(np.uint8))

preprocessor_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/dpt/feature_extraction_dpt.py:30: FutureWarning: The class DPTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DPTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/9.88k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/490M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [5]:
def apply_weather_effect(input_path, prompt, output_path, negative_prompt=None, seed=None):
    base_img = Image.open(input_path).convert("RGB").resize((512, 512))
    depth_map = get_depth_map(base_img).resize((512, 512))

    torch.cuda.empty_cache()
    gc.collect()

    generator = torch.manual_seed(seed) if seed is not None else None

    result = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=base_img,
        control_image=depth_map,
        num_inference_steps=30,
        guidance_scale=4.5,
        controlnet_conditioning_scale=1.2,
        generator=generator
    ).images[0]

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    result.save(output_path)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(np.asarray(base_img))
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(np.asarray(result))
    plt.title("Modified")
    plt.axis("off")
    plt.show()

In [ ]:
apply_weather_effect(
    input_path="/content/bochum_000000_002293_leftImg8bit.png",
    prompt="same exact street scene as input, realistic light snow covering road and sidewalks, trees dusted with snow, cars visible, buildings unchanged, soft natural lighting",
    negative_prompt="cartoon, fantasy, distorted buildings, missing cars, unrealistic textures, snow-covered traffic signs, logo, extra lights, fake roads",
    output_path="outputs/fixed_snow_output.png",
    seed=123,
)


In [ ]:
apply_weather_effect(
    input_path="/content/bochum_000000_002293_leftImg8bit.png",
    prompt="same exact street scene as input, realistic thin fog covering road and sidewalks, light mist hanging in the air, trees partially veiled by fog, all cars clearly visible, buildings unchanged, soft natural diffused lighting, reduced visibility with no change in scene layout",
    negative_prompt="cartoon, fantasy, painting, distorted buildings, altered road layout, added or missing cars, surreal objects, dreamlike effects, text, logos, unrealistic lighting, reflections, color shifts",
    output_path="outputs/foggy_scene_realistic.jpg",
    seed=123
)


In [ ]:
apply_weather_effect(
    input_path="/content/bochum_000000_002293_leftImg8bit.png",
    prompt="enhance the original image with natural daylight, improve brightness and contrast slightly, remove dullness, preserve all original elements exactly — buildings, cars, road, layout, colors, and structure must stay unchanged",
    negative_prompt="cartoon, fantasy, painting, altered buildings, removed cars, added objects, logo, text, surreal colors, style transfer, image distortion",
    output_path="outputs/enhanced_clear_scene.png",
    seed=123,
)


In [ ]:
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt

def brighten_image(input_path, output_path, brightness=2, contrast=1.7, color=1.1):
    img = Image.open(input_path).convert("RGB")

    img = ImageEnhance.Brightness(img).enhance(brightness)
    img = ImageEnhance.Contrast(img).enhance(contrast)
    img = ImageEnhance.Color(img).enhance(color)

    img.save(output_path)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(Image.open(input_path)); plt.title("Original"); plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(img); plt.title("Brightened / Sunny Look"); plt.axis("off")
    plt.show()


In [ ]:
brighten_image(
    input_path="/content/bochum_000000_002293_leftImg8bit.png",
    output_path="outputs/sunny_natural.jpg"
)
